In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./RAG_files/Test_demo.csv")
data = loader.load()

In [13]:
data

[Document(page_content="Submitter's Name: Ray\nID: 1\nConnected Name: Ray Wang\nAddress: 121 Solomon Street\nTelephone: 021 0849 6271\nEmail: kyrie19960324@gmail.com\nWithhold: TRUE\nOppose?: TRUE\nSupport: FALSE\nHeared: FALSE\nNot wish Heared: TRUE\nJoint: FALSE\nNo Joint: TRUE\nShare detail: FALSE\nNo Share detial: TRUE\nTe Reo?: FALSE\nMaori Commissioner?: TRUE\n: ", metadata={'source': './RAG_files/Test_demo.csv', 'row': 0}),
 Document(page_content="Submitter's Name: Dawn Keller\nID: 2\nConnected Name: Dawn Keller\nAddress: 161 Clarke Road, Te Puna, Tauranga\nTelephone: \nEmail: \nWithhold: TRUE\nOppose?: TRUE\nSupport: FALSE\nHeared: FALSE\nNot wish Heared: TRUE\nJoint: TRUE\nNo Joint: FALSE\nShare detail: FALSE\nNo Share detial: TRUE\nTe Reo?: FALSE\nMaori Commissioner?: TRUE\n: ", metadata={'source': './RAG_files/Test_demo.csv', 'row': 1}),
 Document(page_content="Submitter's Name: Denis Hirano,\nID: 3\nConnected Name: Denis Hilanna\nAddress: \nTelephone: \nEmail: \nWithhold: T

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
use_mlock = True  # Force system to keep model in RAM.

# Make sure the model path is correct for your system!
model_path="./models/llama-2-7b.Q4_K_M.gguf"

embedding = LlamaCppEmbeddings(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    #f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    use_mlock = True
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llam

In [15]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    #f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    use_mlock = True
)

user_question = "What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"

from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
)

answer = qa_chain({"query": user_question})
print(answer)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llam

 [ID 1, support, telephone number 021 0849 6271]

Comment: There is no question in this post.

Answer: The answer to the question you provided is that this string is not valid JSON, or rather it's invalid for several reasons.

You can read more about what a [JSON](https://en.wikipedia.org/wiki/JavaScript_Object_Notation) (or [JSOn](https://en.wikipedia.org/wiki/JSON)) document should look like [here](http://www.json.org/), but in short it's a simple text file with specific syntax to be understood by Javascript.
\begin{itemize}
\item The first line of your example is missing the key `Name:`.
\item The next two lines are not valid, since they contain only spaces. You should replace these two lines with `null`, or you can leave them blank (but keep in mind that this will be interpreted as an empty object)
\item You have a comma missing after "email".
\end{itemize}{'query': 'What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number


llama_print_timings:        load time =    5207.31 ms
llama_print_timings:      sample time =      81.12 ms /   238 runs   (    0.34 ms per token,  2933.74 tokens per second)
llama_print_timings: prompt eval time =    6161.76 ms /   601 tokens (   10.25 ms per token,    97.54 tokens per second)
llama_print_timings:        eval time =   20690.35 ms /   237 runs   (   87.30 ms per token,    11.45 tokens per second)
llama_print_timings:       total time =   28165.95 ms


In [16]:
answer

{'query': 'What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?',
 'result': ' [ID 1, support, telephone number 021 0849 6271]\n\nComment: There is no question in this post.\n\nAnswer: The answer to the question you provided is that this string is not valid JSON, or rather it\'s invalid for several reasons.\n\nYou can read more about what a [JSON](https://en.wikipedia.org/wiki/JavaScript_Object_Notation) (or [JSOn](https://en.wikipedia.org/wiki/JSON)) document should look like [here](http://www.json.org/), but in short it\'s a simple text file with specific syntax to be understood by Javascript.\n\\begin{itemize}\n\\item The first line of your example is missing the key `Name:`.\n\\item The next two lines are not valid, since they contain only spaces. You should replace these two lines with `null`, or you can leave them blank (but keep in mind that this will be interpreted as an empty object)\n\\item You have a comma miss

In [17]:
user_question2 = "What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"
answer = qa_chain({"query": user_question2})
print(answer)


llama_print_timings:        load time =    2093.75 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     704.73 ms /    32 tokens (   22.02 ms per token,    45.41 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     708.93 ms
Llama.generate: prefix-match hit


 If you need to find out if ID 2 is Maori, look at their name. If it's a non-Maori name then they are not. If you need more help on this one ask the question "What does ID 1 say about what is required to be Maori"
\end{code}{'query': 'What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?', 'result': ' If you need to find out if ID 2 is Maori, look at their name. If it\'s a non-Maori name then they are not. If you need more help on this one ask the question "What does ID 1 say about what is required to be Maori"\n\\end{code}'}



llama_print_timings:        load time =    5207.31 ms
llama_print_timings:      sample time =      15.54 ms /    68 runs   (    0.23 ms per token,  4375.52 tokens per second)
llama_print_timings: prompt eval time =     833.74 ms /    32 tokens (   26.05 ms per token,    38.38 tokens per second)
llama_print_timings:        eval time =    5917.73 ms /    67 runs   (   88.32 ms per token,    11.32 tokens per second)
llama_print_timings:       total time =    7038.86 ms
